# Trabajo de Laboratorio 4
## Capa de Enlace

In [2]:
from IPython.display import HTML
HTML('''
<style>
.notas {
    background-color: #eeeeee;
    padding: 1em;
    border: 1px solid #aaa;
    margin-top: 1em;
}
</style>
''')


<div class="notas">
Este trabajo práctico puede ser realizado en grupo de hasta **2 integrantes**. Debe entregarse en formato cuadreno (**ipynb**) acompañado de archivos necesarios para su evaluación (imágenes, capturas de paquetes, etc).

Debe contestar en una o más celdas posteriores a las celdas de la consigna. Si modifica algún archivo del práctico, debe hacer primero una copia. Las imágenes en formato SVG pueden ser editadas con el software Inkscape.

Puede ayudarse con este sitio para formatear las tablas https://www.tablesgenerator.com/markdown_tables
</div>


#### Sobre CORE
<div class="notas">
 Para lanzar CORE:
 
 <pre>
 docker run -d \ 
    --hostname coreemu_vnc \
    --name coreemu_vnc \
    --cap-add=NET_ADMIN --cap-add=SYS_ADMIN \
    -p 5901:5900 -p 8080:8080 -p 9999:9999 \
    -v $(pwd):/root/shared 
    unpdit/coreemu_vnc:unp-core-4.8

 </pre>
 
 Luego acceder a http://localhost:8080 para CORE y http://localhost:9999 para Jupyter.
 La carpeta en la que se lanza el contenedor, se comparte como /root/shared, en 
 caso de no funcionar, se puede desde el administrador de archivos de Lab arrastar y soltar
 para subir archivos; y utilizar el botón derecho para descargarlos.
 </br>
 
 En caso de que el contenedor no responda, se puede crear neuvamente:

 `docker start coreemuv_vnc`, `docker stop coreemu_vnc` y `docker restart coreemu_vnc`.
</div>

<div class="notas">
 Alternativamente se puede ejecutar en internet: https://www.youtube.com/watch?v=gkk_KdxShjA
</div>

### Punto 1

En el diagrama de la figura, el usuario en la máquina A ejecuta `ping 172.16.1.10`. En el paquete ICMP dirigido desde A hacia B, visto en la LAN donde está  conectada A, indique cuáles son las direcciones de capa MAC origen  y  destino,  y  las  direcciones  IP  origen  y  destino.  Indique  cómo decidió la máquina A utilizar estas direcciones. Suponga que todos los equipos conocen las direcciones MAC que necesitan.

![](./ej1.svg)

Viendo la comunicación entre los hosts A y B desde la red `172.16.0.0/24`, las direcciones de capa MAC involucradas son las siguientes:

- Origen: MAC-A
- Destino: MAC-R1

Las direcciones IP son las siguientes:

- Origen: 172.16.0.2
- Destino: 172.16.1.10

Como la máquina A desea comunicarse con la máquina B, envía un datagrama IP con dirección destino a `172.16.1.10`. Ahora bien, primero debe pasar por el router R1, así que la máquina A enviará una trama Ethernet a la interfase con MAC `MAC-R1`, para que luego R1 envíe por la interfase con dirección `MAC-R2` otra trama (con mismo payload, pero distinta dirección de destino) hacia la dirección `MAC-B`. 

### Punto 2
Dada la topología de la figura siguiente, responder las siguientes preguntas. Justifique cada una de sus respuestas.
![](./ej2.svg)

a.	Es posible que colisionen dos tramas, una desde Host A a Host C y otra desde Host B a Host D? 

Es posibles que colisiones las tramas de todos los hosts que están por debajo del router (A, B, C, y D) ya que están todos en el mismo **dominio de colisión**

b.	Cuales MAC conoce o puede llegar a conocer el Host B? 

El host B puede llegar a conocer todas aquellas direcciones MAC que estén debajo del router; ellas son:
- Host A
- Host C
- Host D
- Interfase 2 del router

c. El Host C realiza un pedido de ARP (Brodcast de capa2), liste los Host que logran ver el pedido. 

Todos los hosts debajo del router (en la misma subred) reciben el mensaje de broadcast ARP
- Host A
- Host B
- Host D
- Interfase 2 del router

### Punto 3
Dada la topología de la figura siguiente, responder las siguientes preguntas. Justifique cada una de sus respuestas. 
![](./ej3.png)

a. Indicar si la siguiente trama Ethernet puede ser una trama correcta generada en la red de la figura (explicando cuándo y dónde se genera) o no:

| Eth Destino | Eth Origen | Protocolo | IP Origen | IP Destino | .... |
|-------------|------------|-----------|-----------|------------|------|
| b:b:b:b:b:b | a:a:a:a:a:a | IP | 10.1.28.9 | 10.1.74.9 | .... |

No se puede generar. Las MACs son de distintas subredes (el host de origen no conoce la MAC de destino, tiene que pasar por el router R1 para llegar al host B)

b. Indicar qué tipo de trama es la siguiente, para qué se generaría en la red de la figura y cuáles serían los campos relevantes que le faltan::

| Eth Destino | Eth Origen | Protocolo | .... |
|-------------|------------|-----------|------|
| 1:2:3:4:5:6 | a:a:a:a:a:a | ARP | .... |

- Es un datagrama ARP, pero como no está dirigido a la dirección de broadcast, es un paquete de ARP Reply
- El paquete se generaría cuando la interfase del router R1 le contesta con su MAC al host A.
- Los campos relevantes que faltan son los de `IP Origen` e `IP Destino`

c. Indicar la salida en pantalla que genera el comando traceroute que hay que invocar para que aparezca en la red de la figura la siguiente trama:

| Eth Destino | Eth Origen | Protocolo | IP Origen | IP Destino | Protocolo | ..... |
|-------------|------------|-----------|-----------|------------|------|-----|
| b:b:b:b:b:b | 7:8:9:a:b:c | IP | 10.1.28.9 | 10.1.74.9 | UDP  | .... |

En el host A, ejecutar el siguiente comando `traceroute`

```bash
traceroute to 10.1.74.9

1 gateway 10.1.28.1

2 10.1.74.0

3 10.1.74.9
```

### Punto 4
Utilizando la red de la figura anterior, teniendo en cuenta que la máscara de subred es siempre /24, se solicita:

a. Sea un paquete IP con origen en A y destino en B. Si miramos los contenidos de la trama Ethernet en que va encapsulado cuando pasa por la red 10.1.28.0, y los comparamos con los de la trama Ethernet de cuando pasa por la 10.1.74.0, indicar qué campos serán diferentes, tanto de la trama Ethernet como del paquete IP. Señalar, cuando se pueda, el valor que tendría cada uno de esos campos en ambas tramas.

Cuando el paquete IP pasa de una subred a la otra, los campos que cambiarán serán los siguientes:

- Dirección MAC de origen y destino (trama Ethernet)
- TTL (datagrama IP, se decrementará en 1)

Las direcciones MAC serán las siguientes:

- Desde A hacia R1:
    - MAC Origen: `a:a:a:a:a:a`
    - MAC Destino: `1:2:3:4:5:6`
- Desde R1 hacia B:
    - MAC Origen: `7:8:9:a:b:c`
    - MAC Destino: `b:b:b:b:b:b`

b. Con los datos de la figura, indicar en qué subredes Ethernet puede encontrarse un paquete IP con dirección destino 10.1.28.9.

En la subred del host A (`10.1.28.0/24`)

> Pero por qué? No podría encontrarse en cualquier red?

c.	Con los datos de la figura, indicar en qué subredes Ethernet puede encontrarse un paquete IP con dirección origen 10.1.252.9.

d.	Con los datos de la figura, indicar en qué subredes Ethernet puede encontrarse un paquete IP con dirección origen 10.1.254.9 y dirección destino 10.1.253.9.

## Parte 2 - Ethernet

### Punto 5
Dada la siguiente trama Ethernet realice un análisis completo indicando los campos relevantes de la trama y del nivel de red involucrado.
```
00 20 78 e1 5a 80 00 a0 cc 30 c8 db 08 00 45 0b
00 40 f6 00 40 00 80 06 f0 a5 0a 02 00 02 0a 02
00 01 04 1e 00 15 00 4a 19 45 00 00 00 00 b0 02
20 00 e4 38 00 00 02 04 05 b4 01 03 03 00 01 01
08 0a 00 00 00 00 00 00 00 00 01 01 04 02 8f ec
d3 4f
```


### Ethernet

- Destino: `0x002078E15A80`
- Origen: `0x00A0CC30C8DB`
- Tipo: `0x0800` (IPv4)
- FCS (últimos cuatro bytes): `0x8FECD34F`

### IP

- Versión: 4
- Cabecera de 20 bytes de largo (5 words de 32 bits)
- Largo total: `0x0040`
- Banderas:
    - X: 1
    - D: 0
    - M: 0
- Offset: 0
- TTL: `0x80`
- Protocolo: `0x6` (TCP)
- Dirección origen: `10.2.0.2`
- Dirección destino: `10.2.0.1`

### TCP

- Puerto origen: `0x041E` (1054)
- Puerto destino: `0x0015` (21, FTP)
- SEQ: `0x004A1945`
- ACK: `0x00000000`
- Banderas:
    - URG: 0
    - ACK: 0
    - PSH: 0
    - RST: 0
    - SYN: 1
    - FIN: 0
- WIN: `0x2000`


- Datos: desde `0x0103` hasta `0x0402`

### Punto 6
Cree un diagrama en CORE con 2 nodos. Ejectue `brctl show` antes y después de su ejecución. 

a. ¿Cuál es el puente? ¿Qué interfases afecta? 

b. Cree un switch con 3 conexiones. ¿Qué sucede con el bridge?